In [42]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

In [43]:
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable
#np.random.seed(42)

# we generate a four-dimensional random vector 
v = np.random.choice([0, 1], size=4)
print("Random Input is:", v)

# purity of the target state
purity = 1 #Pure State

# choose from 4 target output states: |1010>, |0101>, |0011>, and |1100> based on random v value:
if v[0] == 0 and v[1] == 0:
    bloch_v = Variable(
    torch.tensor([0, 0, 1, 1]),
    requires_grad=False
    )
    print(bloch_v)
if v[0] == 0 and v[1] == 1:
    bloch_v = Variable(
    torch.tensor([0, 1, 0, 1]),
    requires_grad=False
    )
    print(bloch_v)
if v[0] == 1 and v[1] == 0:
    bloch_v = Variable(
    torch.tensor([1, 0, 1, 0]),
    requires_grad=False
    )
    print(bloch_v)
if v[0] == 1 and v[1] == 1:
    bloch_v = Variable(
    torch.tensor([1, 1, 0, 0]),
    requires_grad=False
    )
    print(bloch_v)

# array of Pauli matrices (will be useful later)
Paulis = Variable(torch.zeros([4, 2, 2], dtype=torch.complex128), requires_grad=False)
Paulis[0] = torch.tensor([[0, 1], [1, 0]])
Paulis[1] = torch.tensor([[0, -1j], [1j, 0]])
Paulis[2] = torch.tensor([[1, 0], [0, -1]])

Random Input is: [1 0 0 0]
tensor([1, 0, 1, 0])


In [44]:
# number of qubits in the circuit
nr_qubits = 4
# number of layers in the circuit
nr_layers = 2

# randomly initialize parameters from a normal distribution
params = np.random.normal(0, np.pi, (nr_qubits, nr_layers, 4))
params = Variable(torch.tensor(params), requires_grad=True)

# a layer of the circuit ansatz
def layer(params, j):
    for i in range(nr_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        qml.RZ(params[i, j, 2], wires=i)
        qml.RX(params[i, j, 3], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])

In [45]:
dev = qml.device("default.qubit", wires=4)

In [46]:
@qml.qnode(dev, interface="torch")
def circuit(params, A):

    # repeatedly apply each layer in the circuit
    for j in range(nr_layers):
        layer(params, j)

    # returns the expectation of the input matrix A on the first qubit
    return qml.expval(qml.Hermitian(A, wires=0))

In [47]:
# cost function
def cost_fn(params):
    cost = 0
    for k in range(4):
        cost += torch.abs(circuit(params, Paulis[k]) - bloch_v[k])

    return cost


# set up the optimizer
opt = torch.optim.Adam([params], lr=0.1)

# number of steps in the optimization routine
steps = 200

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = np.zeros((nr_qubits, nr_layers, 3))

print("Cost after 0 steps is {:.4f}".format(cost_fn(params)))

# optimization begins
for n in range(steps):
    opt.zero_grad()
    loss = cost_fn(params)
    loss.backward()
    opt.step()

    # keeps track of best parameters
    if loss < best_cost:
        best_cost = loss
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, loss))

# calculate the Bloch vector of the output state
output_bloch_v = np.zeros(4)
for l in range(4):
    output_bloch_v[l] = circuit(best_params, Paulis[l])

# print results
print("Target Bloch vector = ", bloch_v.numpy())
print("Output Bloch vector = ", output_bloch_v)

Cost after 0 steps is 2.0946
Cost after 10 steps is 0.7068
Cost after 20 steps is 0.7334
Cost after 30 steps is 0.6293
Cost after 40 steps is 0.6464
Cost after 50 steps is 0.6922
Cost after 60 steps is 0.6232
Cost after 70 steps is 0.6221
Cost after 80 steps is 0.6641
Cost after 90 steps is 0.6132
Cost after 100 steps is 0.6225
Cost after 110 steps is 0.6595
Cost after 120 steps is 0.6110
Cost after 130 steps is 0.6222
Cost after 140 steps is 0.6561
Cost after 150 steps is 0.6104
Cost after 160 steps is 0.6210
Cost after 170 steps is 0.6518
Cost after 180 steps is 0.6097
Cost after 190 steps is 0.6197
Cost after 200 steps is 0.6469
Target Bloch vector =  [1 0 1 0]
Output Bloch vector =  [ 0.69808053 -0.03162511  0.71485442  0.        ]
